In [ ]:
# !pip install datasets

In [ ]:
from datasets import load_dataset
import pandas as pd
import spacy
from collections import defaultdict
from typing import Dict
import math
from collections import Counter

1. Use the FIQA-PL dataset that was used in lab 1 and lab lab 2 (so we need the passages, the questions and their relations).


In [ ]:
dataset = load_dataset("clarin-knext/fiqa-pl", 'corpus')
dataset

Generating corpus split: 0 examples [00:00, ? examples/s]

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 57638
    })
})

In [ ]:
df=pd.DataFrame(dataset['corpus'])
df

,_id,title,text
0,3,,"Nie mówię, że nie podoba mi się też pomysł szk..."
1,31,,Tak więc nic nie zapobiega fałszywym ocenom po...
2,56,,Nigdy nie możesz korzystać z FSA dla indywidua...
3,59,,Samsung stworzył LCD i inne technologie płaski...
4,63,,Oto wymagania SEC: Federalne przepisy dotycząc...
...,...,...,...
57633,599946,,">Cóż, po pierwsze, drogi to coś więcej niż hob..."
57634,599953,,"Tak, robią. Na dotacje dla firm farmaceutyczny..."
57635,599966,,">To bardzo smutne, że nie rozumiesz ludzkiej n..."
57636,599975,,„Czy Twój CTO pozwolił dużej grupie użyć „„adm...


In [ ]:
dataset2 = load_dataset("clarin-knext/fiqa-pl-qrels")
df_qa_train = pd.DataFrame(dataset2['train'])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
df_qa_train

,query-id,corpus-id,score
0,0,18850,1
1,4,196463,1
2,5,69306,1
3,6,560251,1
4,6,188530,1
...,...,...,...
14161,11096,407726,1
14162,11097,131224,1
14163,11099,424427,1
14164,11099,150878,1


In [ ]:
dataset3 = load_dataset("clarin-knext/fiqa-pl", "queries")
df_queries = pd.DataFrame(dataset3['queries'])
df_queries['text'][120]

Generating queries split: 0 examples [00:00, ? examples/s]

'Czy czytniki kart kredytowych Square pozwalają na osobisty użytek?'

In [ ]:
df_queries

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online
3,6,,„Dzień roboczy” i „termin płatności” rachunków
4,7,,Nowy właściciel firmy – Jak działają podatki d...
...,...,...,...
6643,4102,,"Jak mogę ustalić, czy moja stopa zwrotu jest „..."
6644,3566,,"Gdzie mogę kupić akcje, jeśli chcę zainwestowa..."
6645,94,,Wykorzystywanie punktów kart kredytowych do op...
6646,2551,,Jak znaleźć tańszą alternatywę dla tradycyjnej...


2. Create a dataset of positive and negative sentence pairs. In each pair the first element is a question and the second element is a passage. Use the relations to mark the positive pairs (i.e. pairs where the question is answered by the passage). Use your own strategy to mark negative pairs (i.e. you can draw the negative examples, but there are better strategies to define the negative examples). The number of negative examples should be much larger than the number of positive examples.

In [ ]:
df_qa_train['query-id'] = df_qa_train['query-id'].astype(str)
df_qa_train['corpus-id'] = df_qa_train['corpus-id'].astype(str)

In [ ]:
results = pd.merge(df_queries, df_qa_train, left_on='_id', right_on='query-id')
results = pd.merge(results, df, left_on='corpus-id', right_on='_id')
new_df = results[['text_x', 'text_y', 'score']]

In [ ]:
new_df = new_df.rename(columns={'text_x': 'query', 'text_y': 'text', 'score': 'class'})
new_df = new_df.sample(frac=1).reset_index(drop=True)
new_df

,query,text,class
0,Superogólny typ funduszu inwestycyjnego,Ponieważ masz już dwukrotnie swój cel w tym fu...,1
1,Czy akcjonariusz może ponosić odpowiedzialność...,„Jestem prawnikiem podatkowym i WSZYSTKIE POWY...,1
2,Jak zaoszczędzić na przewalutowaniu?,Jeśli chcesz przewalutować więcej niż kilka ty...,1
3,Kto ustala cenę i podaje notowane wartości cen...,"„Cena” to cena ostatniej transakcji, która fak...",1
4,"Co się stanie, jeśli zakup w Kanadzie kosztuje...","Jako ktoś, kto pracuje dla firmy, która wdraża...",1
...,...,...,...
14161,"Plusy, minusy i opodatkowanie odszkodowania za...","Uważaj, jeśli ranny w pracy nie będzie dodawał...",1
14162,Dlaczego inwestor długoterminowy miałby kiedyk...,"Widzę kilka powodów, dla których warto rozważy...",1
14163,Czy biznes mojego męża powinien opłacać mój bi...,"Zależy to od zaangażowanych finansów, ale szcz...",1
14164,Możliwy zamiennik Quicken,Czy korzystasz z innych funkcji online Quicken...,1


In [ ]:
new_df['query'][30]

'Dlaczego ludzie zawsze mówią o akcjach, które przynoszą wysokie dywidendy?'

In [ ]:
new_df['text'][30]

'Dlaczego ludzie mówią o akcjach, które przynoszą wysokie dywidendy? Tradycyjnie ludzie, którzy kupują akcje dywidendowe, szukają dochodu ze swoich inwestycji. Większość spółek dywidendowych wypłaca dywidendę co kwartał (co 90 dni). Prawidłowo skonfigurowany inwestor może otrzymywać czek na dywidendę co miesiąc, co tydzień lub tak często, jak ma wystarczająco dużo pieniędzy, aby rozłożyć ex-daty. Istnieje różnica między wysoką kwotą dywidendy a zyskiem. Wypłata dywidendy w wysokości 1 USD na akcję może brzmieć dobrze z góry, ale… ile kosztuje Cię ta akcja? Jeśli akcje kosztują 100 USD za akcję, otrzymujesz 1% zysku. Jeśli akcje kosztują 10 USD za akcję, zyskujesz 10%. Istnieje wiele czynników, które wchodzą w grę przy inwestowaniu w akcje dywidendowe dla przepływu środków pieniężnych. Pamiętaj, dlaczego w ogóle inwestujesz. Wzrost lub przepływ środków pieniężnych. Zorganizuj swoje inwestycje wokół głównych celów inwestycyjnych. Nie ścigaj dużych czeków z dywidendą w dolarach, przeprowa

In [ ]:
(df_qa_train[df_qa_train['corpus-id'] == '131224']['query-id'] == '11097').any()

True

In [ ]:
list(df_qa_train[df_qa_train['query-id'] == str(11099)]['corpus-id'])

['424427', '150878']

In [ ]:
columns = ['query', 'text', 'class']
new_df_2 = pd.DataFrame(columns=columns)

In [ ]:
import random
for query_id in df_qa_train['query-id'].unique():
  corpus_list = list(df_qa_train[df_qa_train['query-id'] == str(query_id)]['corpus-id'])
  for i in range(len(corpus_list)):
    corpus_id = random.choice(df['_id'])
    if str(corpus_id) in corpus_list:
      i-=1
      continue
    new_row = [{'query': df_queries[df_queries['_id'] == str(query_id)]['text'].values[0],'text': df[df['_id'] == str(corpus_id)]['text'].values[0], 'class': 0}]
    new_df_2 = new_df_2.append(new_row, ignore_index=True)



Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
<ipython-input-19-3681af1cf11b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_2 = new_df_2.append(new_row, ignore_index=True)
<ipython-input-19-3681af1cf11b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_2 = new_df_2.append(new_row, ignore_index=True)
<ipython-input-19-3681af1cf11b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_2 = new_df_2.append(new_row, ignore_index=True)
<ipython-input-19-3681af1cf11b>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_2 = new_df_2.append(new_row, ignore_index=True)
<ipython-input-19-3681af1cf11b>

In [ ]:
new_df_2 = new_df_2.sample(frac=1).reset_index(drop=True)

In [ ]:
new_df_2

,query,text,class
0,Dlaczego firmy mają inny rok podatkowy niż kal...,"AMD jest świetne z wielu powodów, z których ni...",0
1,Czy istnieje sposób na uzyskanie danych cen bi...,"Tak, ale jak to się ma do tych z wieloma domam...",0
2,"Jeśli put jest droższy niż połączenie, co to z...",W systemie kanadyjskim nadal istnieją plany ub...,0
3,Czy po konsultacji z HR Block jesteś faktyczni...,Płaca minimalna wzrosła tylko o 2 dolary w cią...,0
4,"Co właściwie kupuję, handlując kontraktami CFD","Podatki od nieruchomości, przynajmniej w Kanad...",0
...,...,...,...
14161,Jak bogaci płacą za rzeczy?,"Nie, reinwestycja odbywa się grzecznościowo. W...",0
14162,Czy mogę oszacować limit kredytowy innych osób...,Możesz rozważyć przekształcenie swojego obecne...,0
14163,Dywidendy – dlaczego naciskamy na reinwestowanie?,"„A Trump powiedział, że chce pomóc małemu bizn...",0
14164,Jak mogę oszacować wartość prywatnych akcji za...,"Ale to jest problem, prawda? Zakładasz sklep n...",0


In [ ]:
final_df = pd.concat([new_df[:1500],new_df_2[:1500]], axis=0)

In [ ]:
final_df = final_df.sample(frac=1).reset_index(drop=True)

In [ ]:
final_df['text'][1]

"„dobre do pieprzenia ich. co się stało z czasami, gdy takich ludzi szanowano? ludzie widywali Rolls Royce'a i mówili „cholera któregoś dnia chcę odnieść taki sukces”. Ludzie kiedyś tak chcieli. wszyscy widzą tego samego rolls-royce'a i myślą: „Co za dupek, powinienem dostać pieniądze, za które odpracował swój tyłek”. Siedząc na kanapie, bezrobotni, brudni i leniwi. Chcesz zmienić te statystyki? i zdobądź swój. nie pozwól, aby ktoś inny kontrolował Twój czas na tym świecie. Życie nie jest sprawiedliwe. Przyzwyczaj się do niego. Chwytaj dzień, wyjdź i spraw, aby Twoje życie było takie, jakie chcesz."

In [ ]:
final_df['text'] = final_df.apply(lambda row: f"Pytanie: {row['query']} Kontekst: {row['text']} Czy kontekst zawiera pytanie?", axis=1)
final_df = final_df.drop(['query'], axis=1)

In [ ]:
final_df['text'][0]


'Pytanie: Kompleksowy samouczek na temat finansów osobistych podwójnego wpisu? Kontekst: Syntetyczny olej do pomp próżniowych może okazać się bardzo skuteczny w zwiększaniu żywotności i wydajności pomp próżniowych. Jest to syntetyczny płyn węglowodorowy, który został dwukrotnie destylowany w celu uzyskania lepszych właściwości. Wysoce zalecane do stosowania w pompach próżniowych do detektorów, może to okazać się bardzo niezawodnym rozwiązaniem. Czy kontekst zawiera pytanie?'

In [ ]:
final_df['class'][0]

0

In [ ]:
final_df

,text,class
0,Pytanie: Kompleksowy samouczek na temat finans...,0
1,Pytanie: Jakie ryzyko wiąże się z pośredniczen...,0
2,Pytanie: Dlaczego nadal istnieją nieelektronic...,0
3,Pytanie: W jaki sposób strony takie jak Google...,1
4,Pytanie: Pranie sprzedaży i implikacje podatko...,0
...,...,...
2995,Pytanie: Najemca chce płacić czynsz za pomocą ...,0
2996,"Pytanie: Jaki wpływ ma mój pracodawca, jeśli m...",1
2997,Pytanie: Jakie podatki są naliczane od dystryb...,0
2998,Pytanie: Podstawy tworzenia zdywersyfikowanego...,0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
final_df.to_csv('dataset.csv', index=False)

In [ ]:
!mkdir gdrive/MyDrive/NLP
!mv dataset.csv gdrive/MyDrive/NLP

!head -30 gdrive/MyDrive/NLP/dataset.csv

mkdir: cannot create directory ‘gdrive/MyDrive/NLP’: File exists
text,class
"Pytanie: Pracownik USA H1B - Zyski kapitałowe w Indiach ze sprzedaży akcji Kontekst: konto Demat, które mam w Indiach, nie jest kontem NRI. Ponieważ nie byłem pewien, jak długo będę w USA, nigdy nie zmieniłem swojego konta na konto NRI. Czy wymagana jest konwersja mojego konta na konto NRI? Tak, jest to bardzo wymagane przez prawo. Nie należy kupować ani sprzedawać akcji na koncie Rezydenta. Należy zamknąć i otworzyć nowe konto Rachunek NRO Demat i przenieść na niego udziały/jednostki. Sprzedaj z tego konta. Jeśli chcesz kupić akcje, gdy jest to NRI, wymagane jest otwarcie konta Demat PINS. Czy kontekst zawiera pytanie?",1
"Pytanie: Czy unikasz podatku biorąc kredyt pod zastaw domu? Kontekst: Dlaczego ktoś miałby inwestować w inne instrumenty (np. akcje), aby opłacić edukację dzieci w college'u, skoro zyski kapitałowe z nich są opodatkowane, w przeciwieństwie do pożyczki pod zastaw domu? Istnieje wiele korzyst